In [1]:
import pandas as pd
import numpy as np

In [8]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
import time

API_KEY = "b26f3e4b3e8d4cd2b2ea5dbc074e76fd"
BASE_URL = "https://api.weatherbit.io/v2.0/history/airquality"

# Kinh độ, vĩ độ Cần Thơ
LON = 106.647339
LAT = 10.839318    # lưu ý: lat 21.x là Hà Nội, Cần Thơ ~10.x

START_DATE = datetime(2023, 1, 1)
END_DATE = datetime(2025, 12, 1)

all_data = []
current = START_DATE

while current < END_DATE:
    # 👉 cộng 1 tháng
    block_end = current + relativedelta(months=1)

    if block_end > END_DATE:
        block_end = END_DATE

    params = {
        "lat": LAT,
        "lon": LON,
        "start_date": current.strftime("%Y-%m-%d"),
        "end_date": block_end.strftime("%Y-%m-%d"),
        "key": API_KEY
    }

    print(f"Fetching data from {params['start_date']} to {params['end_date']}")
    response = requests.get(BASE_URL, params=params)

    if response.status_code == 200:
        json_data = response.json()
        if json_data.get("data"):
            all_data.extend(json_data["data"])
            print(f"Fetched {len(json_data['data'])} records")
        else:
            print("No data found for this period")
    else:
        print(f"Failed: {response.status_code} - {response.text}")
        break

    # 👉 sang tháng tiếp theo (KHÔNG cộng thêm lần nữa)
    current = block_end
    time.sleep(7)

Fetching data from 2023-01-01 to 2023-02-01
Failed: 429 - 


In [ ]:
import pandas as pd

df = pd.DataFrame(all_data)
df.to_csv("cantho_air_quality_HCM.csv", index=False)